There are 4 types of energy to predict <br>
    - 0 : electricity
    - 1 : chilledwater
    - 2 : steam
    - 3 : hotwater

Electricity and water consumption may have different behavior!<br>
     - I will make separately train & predict the model

* Reference 
 - https://www.kaggle.com/corochann/ashrae-training-lgbm-by-meter-type
 - https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction



In [1]:
# No leak data
import pandas as pd
import numpy as np
import gc 
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import mean_squared_error

In [19]:
%%time
# 파일 읽어오기
train_df = pd.read_csv('train.csv')
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_train_df = pd.read_csv('weather_train.csv')


test_df = pd.read_csv('test.csv')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

weather_test_df = pd.read_csv('weather_test.csv')
building_meta_df = pd.read_csv('building_metadata.csv')
sample_submission = pd.read_csv('sample_submission.csv')

Wall time: 58.2 s


In [20]:
#meter_reading ==0 인 값은 없다.
train_df[train_df['meter_reading']<0]
# 1.10 나온 데이터
submission_10 = pd.read_csv('catboost_exp_avg_sub_2019-12-11_08-18-47.csv')
submission_10.head()

,row_id,meter_reading
0,0,209.580186
1,1,103.376081
2,2,12.735707
3,3,241.058419
4,4,526.005096


In [21]:
#site0
site0_df = pd.read_pickle('leakage/site0/site0.pkl')
site0_df['meter_reading'] = site0_df['meter_reading_scraped']
site0_df = site0_df.drop(['meter_reading_scraped','meter_reading_original'],axis =1)
site0_df.loc[site0_df['meter_reading']<0,'meter_reading']=0

site0_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [22]:
# test_df + site0
leak_df = pd.merge(test_df,site0_df,how ='left', on = ['building_id','meter','timestamp'])
leak_df = leak_df.rename(columns = {'meter_reading' :'site0'})
leak_df.head()

,row_id,building_id,meter,timestamp,site0
0,0,0,0,2017-01-01,173.370293
1,1,1,0,2017-01-01,53.512720
2,2,2,0,2017-01-01,6.143042
3,3,3,0,2017-01-01,101.701470
4,4,4,0,2017-01-01,1141.240666


In [23]:
#site1
site1_df = pd.read_pickle('leakage/site1.pkl')
site1_df['meter_reading'] = site1_df['meter_reading_scraped']
site1_df = site1_df.drop(['meter_reading_scraped'],axis =1)
site1_df.loc[site1_df['meter_reading']<0,'meter_reading']=0

site1_df.head()

,building_id,meter,timestamp,meter_reading
0,105,0,2016-01-01 00:00:00,23.303571
1,105,0,2016-01-01 01:00:00,45.607143
2,105,0,2016-01-01 02:00:00,45.607143
3,105,0,2016-01-01 03:00:00,45.107143
4,105,0,2016-01-01 04:00:00,45.607143


In [24]:
# test_df + site1
leak_df = pd.merge(leak_df,site1_df,how ='left', on = ['building_id','meter','timestamp'])
leak_df = leak_df.rename(columns = {'meter_reading' :'site1'})
leak_df.head()

,row_id,building_id,meter,timestamp,site0,site1
0,0,0,0,2017-01-01,173.370293,NaN
1,1,1,0,2017-01-01,53.512720,NaN
2,2,2,0,2017-01-01,6.143042,NaN
3,3,3,0,2017-01-01,101.701470,NaN
4,4,4,0,2017-01-01,1141.240666,NaN


In [25]:
#site2
site2_df = pd.read_csv('leakage/site2/asu_2016-2018.csv')
# site1_df['meter_reading'] = site1_df['meter_reading_scraped']
# site1_df = site1_df.drop(['meter_reading_scraped'],axis =1)
site2_df['timestamp'] = pd.to_datetime(site2_df['timestamp'])
site2_df.loc[site2_df['meter_reading']<0,'meter_reading']=0

site2_df.head()

,timestamp,building_id,meter,meter_reading
0,2016-01-01,156,0,114.71
1,2016-01-01,157,0,28.10
2,2016-01-01,158,0,19.49
3,2016-01-01,159,0,11.82
4,2016-01-01,160,0,5.44


In [26]:
# test_df + site2
leak_df = pd.merge(leak_df,site2_df,how ='left', on = ['building_id','meter','timestamp'])
leak_df = leak_df.rename(columns = {'meter_reading' :'site2'})
leak_df.head()

,row_id,building_id,meter,timestamp,site0,site1,site2
0,0,0,0,2017-01-01,173.370293,NaN,NaN
1,1,1,0,2017-01-01,53.512720,NaN,NaN
2,2,2,0,2017-01-01,6.143042,NaN,NaN
3,3,3,0,2017-01-01,101.701470,NaN,NaN
4,4,4,0,2017-01-01,1141.240666,NaN,NaN


In [27]:
#site 4

site4_df = pd.read_csv('leakage/site4.csv')
site4_df['meter_reading'] = site4_df['meter_reading_scraped']
site4_df = site4_df.drop(['meter_reading_scraped'],axis =1)
site4_df['timestamp'] = pd.to_datetime(site4_df['timestamp'])
site4_df.loc[site4_df['meter_reading']<0,'meter_reading']=0

site4_df.head()

,building_id,timestamp,meter_reading
0,641,2016-01-01 00:00:00,358.548
1,641,2016-01-01 01:00:00,355.919
2,641,2016-01-01 02:00:00,328.179
3,641,2016-01-01 03:00:00,326.403
4,641,2016-01-01 04:00:00,325.490


In [28]:
# test_df + site4
leak_df = pd.merge(leak_df,site2_df,how ='left', on = ['building_id','meter','timestamp'])
leak_df = leak_df.rename(columns = {'meter_reading' :'site4'})
leak_df.head()

,row_id,building_id,meter,timestamp,site0,site1,site2,site4
0,0,0,0,2017-01-01,173.370293,NaN,NaN,NaN
1,1,1,0,2017-01-01,53.512720,NaN,NaN,NaN
2,2,2,0,2017-01-01,6.143042,NaN,NaN,NaN
3,3,3,0,2017-01-01,101.701470,NaN,NaN,NaN
4,4,4,0,2017-01-01,1141.240666,NaN,NaN,NaN


In [29]:
#site 15

site15_df = pd.read_csv('leakage/site15.csv')
# site4_df['meter_reading'] = site4_df['meter_reading_scraped']
site15_df = site15_df.drop(['score'],axis =1)
site15_df['timestamp'] = pd.to_datetime(site15_df['timestamp'])
site15_df.loc[site15_df['meter_reading']<0,'meter_reading']=0

site15_df.head()

,timestamp,meter,meter_reading,building_id
0,2017-01-01 00:00:00,0.0,27.650000,1429
1,2017-01-01 00:00:00,2.0,278.528900,1426
2,2017-01-01 01:00:00,0.0,28.674999,1429
3,2017-01-01 01:00:00,2.0,373.365875,1426
4,2017-01-01 02:00:00,0.0,27.200001,1429


In [30]:
# test_df + site4
leak_df = pd.merge(leak_df,site15_df,how ='left', on = ['building_id','meter','timestamp'])
leak_df = leak_df.rename(columns = {'meter_reading' :'site15'})
leak_df.head()

,row_id,building_id,meter,timestamp,site0,site1,site2,site4,site15
0,0,0,0,2017-01-01,173.370293,NaN,NaN,NaN,NaN
1,1,1,0,2017-01-01,53.512720,NaN,NaN,NaN,NaN
2,2,2,0,2017-01-01,6.143042,NaN,NaN,NaN,NaN
3,3,3,0,2017-01-01,101.701470,NaN,NaN,NaN,NaN
4,4,4,0,2017-01-01,1141.240666,NaN,NaN,NaN,NaN


In [31]:
leak_df['leak_meter_reading'] = leak_df['site0'].fillna(leak_df['site1'].fillna(leak_df['site2']).fillna(leak_df['site4']).fillna(leak_df['site15']))

In [32]:
leak_df= leak_df.drop(['site0','site1','site2','site4','site15'], axis =1 )
leak_df.head()

,row_id,building_id,meter,timestamp,leak_meter_reading
0,0,0,0,2017-01-01,173.370293
1,1,1,0,2017-01-01,53.512720
2,2,2,0,2017-01-01,6.143042
3,3,3,0,2017-01-01,101.701470
4,4,4,0,2017-01-01,1141.240666


In [33]:
leak_df.to_pickle('leak_df.pkl')

In [34]:
%%time
from datetime import datetime
# 1.1 LGBM을 변경해보자
new_submission = pd.concat([submission_10,leak_df['leak_meter_reading']],axis = 1)
new_submission['leak_meter_reading'] = new_submission['leak_meter_reading'].fillna(new_submission['meter_reading'])
new_submission['meter_reading'] = new_submission['leak_meter_reading']
new_submission = new_submission.drop(['leak_meter_reading'],axis =1)
new_submission.head()
new_submission.to_csv('leak_catboost_1.15_' + str(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')) + '.csv', index=False)

Wall time: 1min 51s
